# Web Generator

> Generate a static website from processed documentation

In [ ]:
#| default_exp web_generator

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()
from nbdev.showdoc import *

In [ ]:
#| export
from lovely_docs.utils import settings
from pathlib import Path
import json
import shutil
import markdown
from jinja2 import Environment, FileSystemLoader

In [ ]:
output_dir = settings.output_dir
web_dir = settings.project_root / "web"

print(f"Output directory: {output_dir}")
print(f"Web directory: {web_dir}")

Output directory: ../processed_documents
Web directory: /home/xl0/work/projects/lovely-docs/web


In [ ]:
#| export
def scan_libraries(output_dir: Path) -> dict:
    """Scan the output directory for processed libraries"""
    libraries = {}

    for lib_dir in output_dir.iterdir():
        if not lib_dir.is_dir():
            continue

        index_file = lib_dir / "index.json"
        if not index_file.exists():
            continue

        with open(index_file) as f:
            index_data = json.load(f)

        libraries[lib_dir.name] = {
            "name": index_data.get("source_name"),
            "path": lib_dir,
            "root_directory": index_data.get("root_directory", {}),
            "statistics": index_data.get("statistics", {}),
        }

    return libraries

In [ ]:
libraries = scan_libraries(output_dir)
print(f"Found {len(libraries)} libraries:")
for name, info in libraries.items():
    print(f"  - {name}: {info['statistics'].get('relevant_pages', 0)} pages")

Found 1 libraries:
  - sveltejs-svelte: 9 pages


In [ ]:
#| export
def build_directory_tree(base_path: Path, current_path: Path = None) -> dict:
    """Recursively build a tree structure from the filesystem"""
    if current_path is None:
        current_path = base_path

    metadata_file = current_path / "metadata.json"
    if not metadata_file.exists():
        return None

    with open(metadata_file) as f:
        metadata = json.load(f)

    tree = {
        "name": metadata.get("name"),
        "better_name": metadata.get("better_name"),
        "one_line_summary": metadata.get("one_line_summary"),
        "path": str(current_path.relative_to(base_path)),
        "pages": [],
        "subdirs": []
    }

    # Add pages
    for page_info in metadata.get("pages", []):
        page_name = page_info["better_filename"].rsplit(".", 1)[0].replace(" ", "_")
        page_path = current_path / page_name

        if page_path.exists():
            page_metadata_file = page_path / "metadata.json"
            if page_metadata_file.exists():
                with open(page_metadata_file) as f:
                    page_metadata = json.load(f)

                tree["pages"].append({
                    "name": page_info["better_filename"],
                    "one_line_summary": page_metadata.get("one_line_summary"),
                    "path": str(page_path.relative_to(base_path)),
                })

    # Add subdirectories
    for subdir_info in metadata.get("subdirs", []):
        subdir_name = (subdir_info.get("better_name") or subdir_info["name"]).replace(" ", "_")
        subdir_path = current_path / subdir_name

        if subdir_path.exists():
            subdir_tree = build_directory_tree(base_path, subdir_path)
            if subdir_tree:
                tree["subdirs"].append(subdir_tree)

    return tree

In [ ]:
# Test building tree for sveltejs-svelte
if "sveltejs-svelte" in libraries:
    lib_path = libraries["sveltejs-svelte"]["path"]
    root_name = libraries["sveltejs-svelte"]["root_directory"].get("better_name", "root").replace(" ", "_")
    tree_root = lib_path / root_name

    tree = build_directory_tree(lib_path, tree_root)
    print(json.dumps(tree, indent=2))

{
  "name": "root",
  "better_name": "svelte-reactivity-system",
  "one_line_summary": "Svelte 5 runes are $ -prefixed compiler keywords that control reactivity, state management, derived values, effects, and component binding.",
  "path": "svelte-reactivity-system",
  "pages": [],
  "subdirs": [
    {
      "name": "02-runes",
      "better_name": "svelte-runes-reference",
      "one_line_summary": "Runes are built-in compiler keywords that provide reactive state, derived values, side-effects, props, and debugging utilities for Svelte 5 applications.",
      "path": "svelte-reactivity-system/svelte-runes-reference",
      "pages": [
        {
          "name": "runes-overview.md",
          "one_line_summary": "Runes are $ -prefixed compiler keywords in Svelte that control the compiler behavior, similar to language syntax rather than regular functions.",
          "path": "svelte-reactivity-system/svelte-runes-reference/runes-overview"
        },
        {
          "name": "state-run

In [ ]:
#| export
# Setup Jinja2 environment
web_templates_dir = settings.project_root / "web_templates"
jinja_env = Environment(loader=FileSystemLoader(web_templates_dir))

In [ ]:
#| export
def generate_website(output_dir: Path, web_dir: Path):
    """Generate fully static website from processed documentation"""

    # Clear and create web directory
    if web_dir.exists():
        shutil.rmtree(web_dir)
    web_dir.mkdir(parents=True)

    # Copy JS file from templates (CSS is now from Tailwind CDN)
    shutil.copy2(web_templates_dir / "script.js", web_dir / "script.js")

    # Scan libraries
    libraries = scan_libraries(output_dir)

    # Build trees for each library
    trees = {}
    lib_roots = {}  # Store the root directory names
    for lib_key, lib_info in libraries.items():
        root_name = lib_info["root_directory"].get("better_name", "root").replace(" ", "_")
        tree_root = lib_info["path"] / root_name
        if tree_root.exists():
            trees[lib_key] = build_directory_tree(lib_info["path"], tree_root)
            lib_roots[lib_key] = root_name

    md = markdown.Markdown(extensions=['fenced_code', 'codehilite', 'tables'])

    # Generate library list HTML (absolute paths with Tailwind classes)
    library_list_html = ""
    for lib_key, lib_info in libraries.items():
        library_list_html += f'<a href="/{lib_key}/index.html" class="block px-3 py-2 rounded-md text-sm font-medium text-gray-700 hover:bg-gray-100 hover:text-gray-900 transition-colors">{lib_info["name"]}</a>\n'

    # Generate pages for each library
    for lib_key, lib_info in libraries.items():
        lib_web_dir = web_dir / lib_key
        lib_web_dir.mkdir()

        root_name = lib_roots.get(lib_key)
        source_root = lib_info["path"] / root_name
        tree = trees.get(lib_key)

        # Generate navigation tree HTML (absolute paths, strip root dir from paths)
        nav_tree_html = generate_nav_tree_html(tree, lib_key, root_name)

        # Generate library overview page
        overview_template = jinja_env.get_template("library_overview.html")
        overview_content = overview_template.render(
            name=lib_info["name"],
            one_line_summary=lib_info["root_directory"].get("one_line_summary", ""),
            total_pages=lib_info["statistics"].get("total_pages", 0),
            relevant_pages=lib_info["statistics"].get("relevant_pages", 0),
            relevant_directories=lib_info["statistics"].get("relevant_directories", 0)
        )

        page_template = jinja_env.get_template("page.html")
        overview_html = page_template.render(
            title=lib_info["name"],
            css_path="/style.css",
            js_path="/script.js",
            library_list=library_list_html,
            nav_tree=nav_tree_html,
            content=overview_content
        )

        (lib_web_dir / "index.html").write_text(overview_html)

        # Generate pages for each document (skip root directory level)
        generate_pages_recursive(source_root, source_root, lib_web_dir, lib_key, library_list_html, nav_tree_html, md, root_name)

    # Generate root index.html showing first library (not redirect)
    if libraries:
        first_lib_key = list(libraries.keys())[0]
        first_lib_info = libraries[first_lib_key]
        tree = trees.get(first_lib_key)
        root_name = lib_roots.get(first_lib_key)

        nav_tree_html = generate_nav_tree_html(tree, first_lib_key, root_name)

        overview_template = jinja_env.get_template("library_overview.html")
        overview_content = overview_template.render(
            name=first_lib_info["name"],
            one_line_summary=first_lib_info["root_directory"].get("one_line_summary", ""),
            total_pages=first_lib_info["statistics"].get("total_pages", 0),
            relevant_pages=first_lib_info["statistics"].get("relevant_pages", 0),
            relevant_directories=first_lib_info["statistics"].get("relevant_directories", 0)
        )

        page_template = jinja_env.get_template("page.html")
        root_html = page_template.render(
            title="Lovely Docs",
            css_path="/style.css",
            js_path="/script.js",
            library_list=library_list_html,
            nav_tree=nav_tree_html,
            content=overview_content
        )

        (web_dir / "index.html").write_text(root_html)

    print(f"Website generated at: {web_dir}")
    print(f"Open {web_dir / 'index.html'} in your browser")


def generate_nav_tree_html(tree, lib_key, root_dir_name):
    """Generate navigation tree HTML with absolute paths"""
    if not tree:
        return ""

    nav_content = generate_nav_node_html(tree, lib_key, root_dir_name)

    nav_template = jinja_env.get_template("nav_tree.html")
    return nav_template.render(tree=tree, nav_content=nav_content)


def generate_nav_node_html(node, lib_key, root_dir_name):
    """Generate HTML for a navigation node with absolute paths and Tailwind classes"""
    html = ""

    # Render pages
    for page in node.get("pages", []):
        # Strip the root directory name from the path
        page_path = page['path']
        if page_path.startswith(root_dir_name + "/"):
            page_path = page_path[len(root_dir_name) + 1:]
        elif page_path == root_dir_name:
            page_path = ""

        page_url = f"/{lib_key}/{page_path}/index.html" if page_path else f"/{lib_key}/index.html"
        html += f'<a href="{page_url}" class="block px-3 py-2 rounded-md text-sm text-gray-700 hover:bg-gray-100 hover:text-gray-900 transition-colors" title="{page.get("one_line_summary", "")}">{page["name"]}</a>\n'

    # Render subdirectories
    for subdir in node.get("subdirs", []):
        html += '<div class="mt-4">'
        html += f'<div class="px-3 py-2 text-xs font-semibold text-gray-500 uppercase tracking-wider">{subdir["better_name"] or subdir["name"]}</div>'
        html += '<div class="mt-1 space-y-1">'
        html += generate_nav_node_html(subdir, lib_key, root_dir_name)
        html += '</div></div>'

    return html


def generate_pages_recursive(source_root: Path, current_path: Path, web_root: Path, lib_key: str, library_list_html: str, nav_tree_html: str, md, root_dir_name: str):
    """Recursively generate HTML pages for all documents"""

    page_template = jinja_env.get_template("page.html")
    page_content_template = jinja_env.get_template("page_content.html")

    for item in current_path.iterdir():
        if item.is_dir() and (item / "metadata.json").exists():
            # This is a page or directory
            rel_path = item.relative_to(source_root)
            page_web_dir = web_root / rel_path
            page_web_dir.mkdir(parents=True, exist_ok=True)

            metadata_file = item / "metadata.json"
            with open(metadata_file) as f:
                metadata = json.load(f)

            # Check if this is a page (has summary.md) or directory
            summary_file = item / "summary.md"
            if summary_file.exists():
                # Read markdown files
                summary_md = summary_file.read_text()
                summary_short_md = (item / "summary_short.md").read_text() if (item / "summary_short.md").exists() else ""
                original_md = (item / "original.md").read_text() if (item / "original.md").exists() else ""

                # Convert to HTML
                summary_html = md.convert(summary_md)
                md.reset()
                summary_short_html = md.convert(summary_short_md)
                md.reset()
                original_html = md.convert(original_md)
                md.reset()

                # Generate page content
                page_content = page_content_template.render(
                    title=metadata.get("better_filename", metadata.get("better_name", metadata.get("name"))),
                    one_line_summary=metadata.get("one_line_summary", ""),
                    summary_html=summary_html,
                    summary_short_html=summary_short_html,
                    original_html=original_html
                )

                # Generate full page with absolute paths
                page_html = page_template.render(
                    title=metadata.get("better_filename", metadata.get("better_name", metadata.get("name"))),
                    css_path="/style.css",
                    js_path="/script.js",
                    library_list=library_list_html,
                    nav_tree=nav_tree_html,
                    content=page_content
                )

                (page_web_dir / "index.html").write_text(page_html)

            # Recurse into subdirectories
            generate_pages_recursive(source_root, item, web_root, lib_key, library_list_html, nav_tree_html, md, root_dir_name)

In [ ]:
# Generate the website
generate_website(output_dir, web_dir)

Website generated at: /home/xl0/work/projects/lovely-docs/web
Open /home/xl0/work/projects/lovely-docs/web/index.html in your browser


In [ ]:
import subprocess
import os

In [ ]:
# Serve the website with a simple HTTP server


print(f"Starting server at http://localhost:8000")
print(f"Serving from: {web_dir}")
print("Press Ctrl+C to stop")

os.chdir(web_dir)
subprocess.run(["python", "-m", "http.server", "8000"])

Starting server at http://localhost:8000
Serving from: /home/xl0/work/projects/lovely-docs/web
Press Ctrl+C to stop
Serving HTTP on 0.0.0.0 port 8000 (http://0.0.0.0:8000/) ...


127.0.0.1 - - [24/Oct/2025 19:51:06] "GET /sveltejs-svelte/svelte-runes-reference/state-rune-guide/index.html HTTP/1.1" 200 -
127.0.0.1 - - [24/Oct/2025 19:51:06] "GET /script.js HTTP/1.1" 200 -



Keyboard interrupt received, exiting.


KeyboardInterrupt: 